In [2]:
!pip install langchain langchain_community
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install bitsandbytes

In [3]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'xxxxxxxxxxxxxxxxxxxxx'

# Access Model on Hugging Face Hub

In [4]:
prompt = PromptTemplate(
    input_variables=['product'],
    template = 'What is a good name for a company that makes {product}')

In [46]:
llm = HuggingFaceHub(repo_id='google/flan-t5-large', model_kwargs={'temperature':0, 'max_length':64})

In [22]:
chain = LLMChain(llm=llm, prompt=prompt)

In [23]:
chain.invoke('colorful socks')

{'name': 'colorful socks', 'text': 'Marvel Comics'}

In [28]:
prompt = PromptTemplate(
    input_variables=['name'],
    template = 'can you tell me about famous marvel character {name}')

In [29]:
chain = LLMChain(llm=llm, prompt=prompt)

In [30]:
chain.invoke('iron man'),

({'name': 'iron man', 'text': 'Iron Man is a Marvel superhero'},)

In [35]:
prompt = PromptTemplate(
    input_variables=['cuisine'],
    template = 'Can you tell me food items for a  {cuisine} restaurant')

In [36]:
chain = LLMChain(llm=llm, prompt=prompt)

In [37]:
chain.invoke('indian')

{'cuisine': 'indian', 'text': 'Indian food'}

# Download Model Locally - Create Pipeline

In [50]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [51]:
model_id = 'google/flan-t5-large'

In [53]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [54]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='cuda')

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [56]:
pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer, max_length=128)

In [57]:
local_llm = HuggingFacePipeline(pipeline=pipeline)

/tmp/ipykernel_64/1286752615.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipeline)


In [61]:
prompt = PromptTemplate(
    input_variables=['product'],
    template = 'What is a good name for a company that makes {product}')

chain = LLMChain(llm=local_llm, prompt=prompt)
chain.invoke('colorful socks')

{'product': 'colorful socks', 'text': 'sock mania'}

In [65]:
prompt = PromptTemplate(
    input_variables=['cuisine'],
    template = 'Can you tell me restaurant name for a  {cuisine} restaurant')

chain = LLMChain(llm=local_llm, prompt=prompt)
chain.invoke('fried chicken')

{'cuisine': 'fried chicken', 'text': 'The Chicken House'}

In [78]:
prompt = PromptTemplate(
    input_variables=['name'],
    template = 'Can you tell me one punch dialogue of {name}')

chain = LLMChain(llm=local_llm, prompt=prompt)
chain.invoke('iron man')

{'name': 'iron man', 'text': 'he slams the door'}